In [1]:
import arcpy
import requests
import zipfile
import io
from arcpy import env

In [2]:
# Sets workspace
arcpy.env.workspace = r'E:\ArcGIS_1\Lab2\LiDAR'
workspace = arcpy.env.workspace

## Elevation

In [3]:
# Downloads .las file and saves to disk
lidar_link = r'https://resources.gisdata.mn.gov/pub/data/elevation/lidar/examples/lidar_sample/las/4342-12-05.las'
lidar_output = requests.post(lidar_link)
with open(r'MGC.las', 'wb') as las:
    las.write(lidar_output.content)

In [4]:
# Creates LAS Dataset
arcpy.management.CreateLasDataset(r"MGC.las", r"MGC_LasDataset.lasd", "NO_RECURSION", None, 'PROJCS["datum_D_North_American_1983_HARN_UTM_Zone_15N",GEOGCS["GCS_datum_D_North_American_1983_HARN",DATUM["D_unknown",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["false_easting",500000.0],PARAMETER["false_northing",0.0],PARAMETER["central_meridian",-93.0],PARAMETER["scale_factor",0.9996],PARAMETER["latitude_of_origin",0.0],UNIT["Meter",1.0]],VERTCS["NAVD88 - Geoid03 (Meters)",VDATUM["unknown"],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]]', "COMPUTE_STATS", "ABSOLUTE_PATHS", "NO_FILES", "DEFAULT", None, "INTERSECTED_FILES")

<Result 'E:\\ArcGIS_1\\Lab2\\LiDAR\\MGC_LasDataset.lasd'>

In [5]:
# Creates TIN y DEM
arcpy.ddd.LasDatasetToTin(r"MGC_LasDataset.lasd", r"MGC_TIN", "WINDOW_SIZE", "MIN", 13.989, 500000, 10, "CLIP")
arcpy.conversion.LasDatasetToRaster(r"MGC_LasDataset.lasd", r"MGC_DEM.tif", "ELEVATION", "BINNING AVERAGE LINEAR", "FLOAT", "CELLSIZE", 10, 1)

<Result 'E:\\ArcGIS_1\\Lab2\\LiDAR\\MGC_DEM.tif'>

In [6]:
# Creates reference to map
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.listMaps("Map")[0]

# Creates DEM layer for display in layout
arcpy.management.SaveToLayerFile("MGC_DEM.tif", r"MGC_DEM_layer.lyrx", None, "CURRENT")

# Reference to layout 
mgc_dem = arcpy.mp.LayerFile(r"E:\ArcGIS_1\Lab2\LiDAR\MGC_DEM_layer.lyrx")
m.addLayer(mgc_dem)
lyt = aprx.listLayouts("Layout")[0]

# Exports DEM layout
lyt.exportToPDF(r"mgc_dem.pdf")

'mgc_dem.pdf'

In [7]:
# Creates TIN layer for display in layout
arcpy.management.SaveToLayerFile("MGC_TIN", r"MGC_TIN_layer.lyrx", None, "CURRENT")

# Reference to layout
mgc_dem = arcpy.mp.LayerFile(r"E:\ArcGIS_1\Lab2\LiDAR\MGC_TIN_layer.lyrx")
m.addLayer(mgc_dem, 'TOP')
lyt = aprx.listLayouts("Layout")[0]

# Exports TIN layout
lyt.exportToPDF(r"mgc_tin.pdf")

'mgc_tin.pdf'

## Precipitation

In [8]:
# New folder for the .bil files
arcpy.CreateFolder_management(workspace, "PRISM")

<Result 'E:\\ArcGIS_1\\Lab2\\LiDAR\\PRISM'>

In [9]:
# Extracts dataset from PRISM API
prism_link = r'https://prism.oregonstate.edu/fetchData.php?type=all_bil&kind=normals&spatial=4km&elem=ppt&temporal=annual'
prism_output = requests.post(prism_link)
zipfile.ZipFile(io.BytesIO(prism_output.content)).extractall('PRISM')

In [10]:
# Creates mosaic dataset in the default geodatabase
sr = arcpy.SpatialReference(3857)
empty_mosaic = arcpy.management.CreateMosaicDataset(r"E:\ArcGIS_1\Lab2\LiDAR\LiDAR.gdb", "PRISM", sr)

In [11]:
# Adds the monthly bil rasters to the mosaic dataset
arcpy.management.AddRastersToMosaicDataset("PRISM", "Raster Dataset", r"E:\ArcGIS_1\Lab2\LiDAR\PRISM\PRISM_ppt_30yr_normal_4kmM3_01_bil.bil;E:\ArcGIS_1\Lab2\LiDAR\PRISM\PRISM_ppt_30yr_normal_4kmM3_02_bil.bil;E:\ArcGIS_1\Lab2\LiDAR\PRISM\PRISM_ppt_30yr_normal_4kmM3_03_bil.bil;E:\ArcGIS_1\Lab2\LiDAR\PRISM\PRISM_ppt_30yr_normal_4kmM3_04_bil.bil;E:\ArcGIS_1\Lab2\LiDAR\PRISM\PRISM_ppt_30yr_normal_4kmM3_05_bil.bil;E:\ArcGIS_1\Lab2\LiDAR\PRISM\PRISM_ppt_30yr_normal_4kmM3_06_bil.bil;E:\ArcGIS_1\Lab2\LiDAR\PRISM\PRISM_ppt_30yr_normal_4kmM3_07_bil.bil;E:\ArcGIS_1\Lab2\LiDAR\PRISM\PRISM_ppt_30yr_normal_4kmM3_08_bil.bil;E:\ArcGIS_1\Lab2\LiDAR\PRISM\PRISM_ppt_30yr_normal_4kmM3_09_bil.bil;E:\ArcGIS_1\Lab2\LiDAR\PRISM\PRISM_ppt_30yr_normal_4kmM3_10_bil.bil;E:\ArcGIS_1\Lab2\LiDAR\PRISM\PRISM_ppt_30yr_normal_4kmM3_11_bil.bil;E:\ArcGIS_1\Lab2\LiDAR\PRISM\PRISM_ppt_30yr_normal_4kmM3_12_bil.bil", "UPDATE_CELL_SIZES", "UPDATE_BOUNDARY", "NO_OVERVIEWS", None, 0, 1500, None, '', "SUBFOLDERS", "ALLOW_DUPLICATES", "BUILD_PYRAMIDS", "CALCULATE_STATISTICS", "NO_THUMBNAILS", '', "NO_FORCE_SPATIAL_REFERENCE", "ESTIMATE_STATISTICS", None, "NO_PIXEL_CACHE", r"C:\Users\osori050\AppData\Local\ESRI\rasterproxies\PRISM")

<Result 'PRISM'>

In [12]:
# Creates two new fields: i) a colum populated with the word Precipitation
# and i) a column with dates for time series
arcpy.management.CalculateField(r"PRISM\Footprint", "Variable", '"Precipitation"', "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField(r"PRISM", "Timestamp", 'DateAdd(Date(2017, 0, 1), $feature.OBJECTID-1, "month")', "ARCADE", '', "DATE", "NO_ENFORCE_DOMAINS")

<Result 'PRISM'>

In [13]:
# Builds multidimensional info to the mosaic dataset to create then
# a multidimensional raster layer 
arcpy.md.BuildMultidimensionalInfo("PRISM", "Variable", "Timestamp # Date", "Precipitation # #", "NO_DELETE_MULTIDIMENSIONAL_INFO")
arcpy.md.MakeMultidimensionalRasterLayer("PRISM", "PRISM_data_MultidimLayer", "Precipitation", "ALL", None, None, '', '', '', None, '', '-125.020833333333 24.0624999997935 -66.4791666661985 49.9375000000005 GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "DIMENSIONS", None)

<Result 'PRISM_data_MultidimLayer'>

In [14]:
# Creates Space Time Cubes
arcpy.stpm.CreateSpaceTimeCubeMDRasterLayer("PRISM_data_MultidimLayer", r"precipitation_cube.nc", "ZEROS")

<Result 'E:\\ArcGIS_1\\Lab2\\LiDAR\\precipitation_cube.nc'>

In [15]:
# Visualization in 3D of the Space Time cubes
arcpy.stpm.VisualizeSpaceTimeCube3D(r"precipitation_cube.nc", "PRECIPITATION_NONE_ZEROS", "VALUE", r"precipitation_cube_VisualizeSpaceTimeCube3D")

<Result 'E:\\ArcGIS_1\\Lab2\\LiDAR\\precipitation_cube_VisualizeSpaceTimeCube3D.shp'>